# 神经网络的搭建

深度学习案例的4个步骤：
1. 准备数据
2. 搭建神经网络
3. 模型训练
4. 模型测试

搭建神经网络的流程
1. 定义一个类，继承自`nn.Module`
2. 在类的初始化方法__init__()中定义神经网络的层
3. 实现`forward`方法，定义前向传播的计算过程

接下来我们来构建如下图所示的神经网络模型：

<img src="./docs/02-神经网络基础/assets/image-20220313133601678.png" alt="image-20220313133601678" style="zoom:50%;" />

**编码设计如下：**

- 第1个隐藏层：权重初始化采用标准化的xavier初始化 激活函数使用sigmoid
- 第2个隐藏层：权重初始化采用标准化的HeKaiming初始化 激活函数采用relu
- out输出层线性层 假若多分类，采用softmax做数据归一化

In [ ]:
"""
搭建神经网络

下载torchsummary，用来计算模型参数，查看模型结构
pip install torchsummary
"""

import torch
import torch.nn as nn
from torchsummary import summary

# 1. 搭建神经网络，自定义继承nn.Module
class ModuleDemo(nn.Module):
    def __init__(self):
        # 1.1 初始化父类成员
        super().__init__()
        # 1.2 搭建神经网络 --> 隐藏层 + 输出层
        # 隐藏层1：输入特征数3，输出特征数3
        self.linear1 = nn.Linear(3, 3)
        # 隐藏层2：输入特征数3，输出特征数2
        self.linear2 = nn.Linear(3, 2)
        # 输出层：输入特征数2，输出特征数2
        self.output = nn.Linear(2, 2)

        # 1.3 对隐藏层进行参数初始化
        # 隐藏层1
        nn.init.xavier_normal_(self.linear1.weight)
        nn.init.zeros_(self.linear1.bias)

        # 隐藏层2
        nn.init.kaiming_normal_(self.linear2.weight)
        nn.init.zeros_(self.linear2.bias)

        # 输出层

    # 2.前向传播
    def forward(self, x):
        # 2.1 第一层，隐藏层计算：加权求和 + 激活函数sigmoid
        # 分解版写法
        # x = self.linear1(x)  # 加权求和
        # x = torch.sigmoid(x)  # 激活函数

        # 合并版写法
        x = torch.sigmoid(self.linear1(x))  # 加权求和 + 激活函数

        # 2.2 第二层，隐藏层计算：加权求和 + 激活函数relu
        x = torch.relu(self.linear2(x)) 

        # 2.3 第三层，输出层计算：加权求和
        # dim=-1 表示对最后一个维度进行softmax操作
        x = torch.softmax(self.output(x), dim=-1)

        # 2.4 返回预测值
        return x

In [29]:
# 3.模型训练
def train():
    my_module = ModuleDemo()
    print(f"my_module: {my_module}")
    # 创建数据集
    data = torch.randn(size=(5, 3))
    print(f"data: {data}")

    # 调用神经网络模型 -> 进行模型训练
    output = my_module(data)  # 底层自动调用forward方法，进行前向传播
    print(f"output: {output}")
    print(f"output shape: {output.shape}")  # (5,2)
    print(f"output required_grad: {output.requires_grad}")  # True

    # 计算和查看模型参数
    print('==============计算模型参数==============')
    # 参1：模型对象
    # 参2：输入数据维度（5行3列）
    summary(my_module, input_size=(5, 3))
    print("==============查看模型参数==============")
    # 查看模型参数
    for name, param in my_module.named_parameters():
        print(f"参数名: {name}")
        print(f"参数值: {param}\n")

if __name__ == "__main__":
    train()

my_module: ModuleDemo(
  (linear1): Linear(in_features=3, out_features=3, bias=True)
  (linear2): Linear(in_features=3, out_features=2, bias=True)
  (output): Linear(in_features=2, out_features=2, bias=True)
)
data: tensor([[-0.5950, -0.0918, -0.1226],
        [-0.8373,  0.9182, -0.2404],
        [-0.2893, -0.8418, -1.7544],
        [ 0.2223,  1.3177,  1.6061],
        [-0.1040, -0.5004,  1.6244]])
output: tensor([[0.6701, 0.3299],
        [0.6780, 0.3220],
        [0.6712, 0.3288],
        [0.6731, 0.3269],
        [0.6584, 0.3416]], grad_fn=<SoftmaxBackward0>)
output shape: torch.Size([5, 2])
output required_grad: True
==============计算模型参数==============
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 5, 3]              12
            Linear-2                 [-1, 5, 2]               8
            Linear-3                 [-1, 5, 2]               6
Total params: 2